# Kaggle Bike Share competition
Background here is that I tried using this bike share script I had written to run linear regression predicting the counts of people taking out bikes from a bike share hourly. I did a bunch of data preprocessing based on what I was learning in PoDS, and then I tried using the Microsoft Azure ML tutorials to port this over to Azure ML. However, trying to be too ambitious, I changed too many things, and when I predict I get ridiculous values. I also coded it stupidly so that all my debugging couldn't be done with each run. 

I'm retrying it all!

I will start by getting the Workspace in Azure

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()
#alternative Workspace.get (name=, subscription_id=, resource_group=)

In [2]:
type(ws)

azureml.core.workspace.Workspace

In [3]:
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name , ':', compute.type)

myCIWorkstationRyan : ComputeInstance
myCClusterRyan : AmlCompute


Below I start with defining a function that
1. Converts the `datetime` feature into a nominal "time of day" feature, then converts that into dummy variables
2. Converts nominal feature `season` into dummy variables
3. Converts nominal feature `weather` into dummy variables

In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


def when_is_it(hour):
    if hour >=5 and hour < 11:
        return 'morning'
    elif hour >=11 and hour < 16:
        return 'afternoon'
    elif hour >=16 and hour < 18:
        return 'rush_hour'
    else:
        return 'off_hours'

def season_is_it(season_int):
    if season_int == 1:
        return 'spring'
    elif season_int == 2:
        return 'summer'
    elif season_int == 3:
        return 'fall'
    else:
        return 'winter'
    
def weather_is_it(weather_int):
    if weather_int == 1:
        return 'nice'
    elif weather_int == 2:
        return 'misty'
    elif weather_int == 3:
        return 'ugly'
    else:
        return 'stormy'
    
#function takes all nominal features and converts to dummy features
def dummy_conversion(data): 
    #create the rules for each dummy variable
    df_when_is_it = data['when_is_it'].apply(when_is_it)
    df_season_is_it = data['season'].apply(season_is_it)
    df_weather_is_it = data['weather'].apply(weather_is_it)
    
    
    when_dummies = pd.get_dummies(df_when_is_it, prefix = 'when_')    
    season_dummies = pd.get_dummies(df_season_is_it, prefix = 'season_')
    weather_dummies = pd.get_dummies(df_weather_is_it, prefix = 'weather_')
    
    #drop the old nominal veriables
    data=data.drop('datetime', axis = 1 )
    data=data.drop('season', axis = 1)
    data=data.drop('weather', axis = 1)
    
    data[list(when_dummies.columns)] = when_dummies
    data[list(season_dummies.columns)] = season_dummies
    data[list(weather_dummies.columns)] = weather_dummies
    return data                 

## Step 1: Create Experiment
 remember toimport Experiment from `azureml.core`

In [5]:
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name='bike-sharing-demand-LinRegBest')

## Step 2: Load data and prepare for training


Load data below

In [6]:
bikes = pd.read_csv('train.csv')

test_data = pd.read_csv('test.csv')
temp_datetime = pd.read_csv('test.csv')['datetime']

feature_cols = ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed']    

y = bikes['count']
bikes=bikes[feature_cols]

bikes['when_is_it']= bikes['datetime'].apply(lambda x:int(x[11]+x[12]))
test_data['when_is_it'] = test_data['datetime'].apply(lambda x:int(x[11]+x[12]))
                      



X=dummy_conversion(bikes)
test_data=dummy_conversion(test_data)

#Apply scaler to the features - I was told this helps with many algorithms - it seemed to help out here
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

Split Data below

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
    #X_train[feature_cols] = scaler.transform(X_train[feature_cols])
    #X_test[feature_cols] = scaler.transform(X_test[feature_cols])
    #X_train and y_train will be used to train the model
    #X_test and y_test will be used to test the model
    #remember that all four of these variables are just subsets of the overall X and Y

## Step 3: Train a model

Following design pattern shows how to leverage the SDK to easily keep track of your training in the cloud.

In [84]:
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



#create the run
#tutorial usese this to start the experiment, run.get_context() is used in MS Learn
run = experiment.start_logging()

model = LinearRegression()
    #instantiate the model

model.fit(X_train, y_train)
    # fit the model to our training set

y_pred = model.predict(X_test)
    #predict our testing set
    #y_test = y_test.abs()
    
    #the log error doesn't take negative values, linear regression however can predict negative counts.
    #since this makes no sense, I make any negative predictions = 0.
y_pred[y_pred <0] = 0
    
error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
run.log('RMSLE', error)
    #error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    # calculate our metric
    
print('RMSLE is ' + str(error))

run.complete()



RMSLE is 1.177293425470474


In [12]:
scaler.fit(test_data)
test_data=scaler.transform(test_data)

predictions = linreg.predict(test_data)
#ensure all negative values predict 0
predictions[predictions <0] = 0 

#ensure all counts are rounded
predictions = np.rint(predictions)
submission = pd.DataFrame({'datetime': temp_datetime, 'count': predictions})
submission.to_csv('linReg_submission2', index=False, )

`11:06 Aug 1: ` So here's a fun observation, when I was doing up this notebook, I had the code separated out in such a way that I would run the code above and the RMSLE wouldn't change. And I was like... What? I examined above, and the X and Y split happened earlier, and it appears the RMSLE is varying due to the randomness in the training/test split.

This inspired me to go back above and change up my code to test out 18 (20-1 splits), here I will show it below!


In [7]:
import joblib
#create the run

test_splits = [0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.5]

best_split = None
best_split_error=None
#we are running an experiment where we are testing different test sizes, since train_test_split randomized, we test 20 different splits (arbitrarily chosen) to find the lowest 
#RMSLE, once we find the lowest among the 20 we log it and 
for split in test_splits:
    run = experiment.start_logging()
    run.log('test-size: ', split)
    model = LinearRegression()
    min_error=None
    
    for i in range(20):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)
        
        
        
        model.fit(X_train, y_train)
        # fit the model to our training set

        y_pred = model.predict(X_test)
        #predict our testing set
        #y_test = y_test.abs()
    
        #the log error doesn't take negative values, linear regression however can predict negative counts.
        #since this makes no sense, I make any negative predictions = 0.
        y_pred[y_pred <0] = 0
        
        
        current_error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
        #checks if there's a min error, if there is, set to lowest of current and previous iteration
        if min_error==None:
            min_error = current_error
        else:
            if min_error > current_error:
                min_error=current_error
    run.log('RMSLE', min_error)
    
    if best_split_error== None:
        best_split_error=min_error
        best_split = split
    else:
        if best_split_error > min_error:
            best_split_error = min_error
            best_split = split
        #error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
        # calculate our metric
    print('test size: ' + str(split) + 'RMSLE is ' + str(min_error))
    run.complete()
    
    model_name = "model_split_" + str(split) + ".pkl"
    filename = "outputs/" + model_name

    joblib.dump(value=model, filename=filename)
    run.upload_file(name=model_name, path_or_stream=filename)
    
print('best (split, RMSLE)')
print(best_split, best_split_error)



test size: 0.05RMSLE is 1.073977492798418
test size: 0.1RMSLE is 1.109083093933371
test size: 0.15RMSLE is 1.1194627874171397
test size: 0.2RMSLE is 1.1626191128813523
test size: 0.25RMSLE is 1.1373476297400904
test size: 0.3RMSLE is 1.137103137328353
test size: 0.35RMSLE is 1.138879107572037
test size: 0.4RMSLE is 1.1551912703890983
test size: 0.45RMSLE is 1.1713541647048589
test size: 0.5RMSLE is 1.1575844389352992
best (split, RMSLE)
0.05 1.073977492798418


In [8]:
#loads the saved model
#fits test data to scaler
scaler.fit(test_data)
test_data=scaler.transform(test_data)

for split in test_splits:
    model = joblib.load('model_split_' + str(split) + '.pkl')
    predictions = model.predict(test_data)
    #ensure all negative values predict 0
    predictions[predictions <0] = 0 

    #ensure all counts are rounded
    predictions = np.rint(predictions)
    submission = pd.DataFrame({'datetime': temp_datetime, 'count': predictions})
    submission.to_csv('linReg_submission_'+ str(split) + '.csv', index=False, )

FileNotFoundError: [Errno 2] No such file or directory: 'model_split_0.1.pkl'

After submitting, I actually got slightly worse than last time.

NameError: name 'best_run' is not defined

https://www.kaggle.com/c/bike-sharing-demand
Above is the link I submit to. I officially got 1.46 RMSLE on the site when I submitted without dummy variables, but included scaling. 

My new submission... GOT 1.199984 = 1.2 RMSLE WOOOOOOOOO